# Laboratorio 5: Clasificación 🤗

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>


### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Esteban Araneda
- Nombre de alumno 2: Ignacio Reyes

### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/...../)

### Temas a tratar
- Clasificación en problemas desbalanceados
- Lightgbm y xgboost
- Pipelines

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.





### Objetivos principales del laboratorio
- Comprender cómo trabajar con problemas de clasificación con clases desbalanceadas.
- Aplicar los modelos lightgbm y xgboost.
- Practicar Pipelines

# Parte Teórica [12 puntos]




1. Explique cuál es la diferencia entre los datos de entrenamiento y validación. [1 punto]

2. Explique cuál es el principal desafío al trabajar problemas de clasificación con data no supervisada. [1 punto]

3. Explique en **sus palabras** qué es la matriz de confusión y para qué se utiliza. [1 puntos]

4. Escriba la fórmula de las siguientes métricas y explique con **sus palabras** cómo se interpretan. [1 punto cada uno]

  * Accuracy
  * Precision
  * Recall
  * F1 score

5. Explique qué métrica recomendaría para los siguientes contextos de clasificación. [1 punto cada uno]

  * Mantenimiento predictivo de fallas de maquinaria pesada en la industria minera.  
  * Detección de enfermedades altamente contagiosas.
  * Aprobación de créditos de alto riesgo.
  * Detección de crímenes.

6. Explique qué es la calibración de modelos y para qué se usa. [1 punto]


**Respuesta:**

*Escriba su respuesta aquí*

1) El conjunto de entrenamiento se ocupa para entrenar el modelo, el conjunto de validación se ocupa para revisar el desempeño del modelo sin tocar los datos de prueba.

2) Cómo no son datos etiquetados, en general, no se puede validar  que los clusters encontrados corresponden a una separado o relación real de los datos. Esto también da problemas cuando hay datos solapados que incluso en clasificación supervisada darían problemas.
3) La matriz de confución reune la información de la cantidad (o porcentaje si se normaliza) de los aciertos y fallos por clase de un modelo. La utilidad es que permite conocer la cantidad relativa  en que se espera que el modelo acierte o falle de clasificar correctamente dentro de un conjunto de datos nuevo.

4) Accuracy : $ \frac{n°predicciones correctas}{n°predicciones totales} $ Se interpreta como la fracción de acierto general del modelo.

Precision :  $ \frac{TP}{TP+FP} $ Se interpreta como la confianza que tenemos del modelo en la clasificación positiva.

Recall:  $ \frac{TP}{TP+FN} $ Se interpreta como seguridad que tenemos sobre el modelo en identificar un elemento de clase positiva.

F1-Score:  $ \frac{2*P*R}{P+R}  $ Es interpretada como una comparación entre los true positive y ambos errores.

 5)
 - Minería: Recall, no se quiere que haya fallas por falta de mantención en aquellas que la necesitan.
 - Enfermedad: Recall, no queremos falsos negativos que contagien.
 - Crédito: Precision, el aprobar una persona que no pagará el credito es regiesgoso.
 - Crímenes: F1 score, no se quiere dejar al criminal libre, pero es inaceptable el encarcelar un inocente.

 6) La calibración consiste en corregir los sesgos del modelo comparando las probabilidades de que un elemento  pertenesca a una clase predicha u otra  vs una probabilidad real. Se usa para autocontener el modelo (sesgos del modelo no deben influir en pasos futuros) y estimar los sesgos ante un dato de clasificación incierto.

# Parte práctica [48 puntos]

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones.

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con el csv `statsplayers.csv`, donde deberán aplicar algoritmos de aprendizaje supervisado de clasificación en base a características que describen de jugadores de fútbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [1]:
# # Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
# try:
#     from google.colab import drive
#     drive.mount("/content/drive")
#     path = 'Dirección donde tiene los archivos en el Drive'
# except:
#     print('Ignorando conexión drive-colab')

## 1. Predicción de Seleccionados Nacionales [14 puntos]

<p align="center">
  <img src="https://www.futuro.cl/wp-content/uploads/2016/06/chile-argentina-meme-12.jpg" width="300">
</p>



### 1.1 Preprocesamiento [5 puntos]

Tareas:

1. Genere los labels para la clasificación binaria en una variable llamada `label`. Para esto, trabaje sobre el atributo `National_Position` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. [Sin puntaje]

2. Hecho esto, ¿cuántos se tienen ejemplos por cada clase? Comente lo que observa. [1 punto]

3. Genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guárdelo una variable llamada `col_transformer`. [2 puntos]

4. Comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material `Player_Stats_Report.html` que viene en el zip del lab), al igual que las transformaciones aplicadas. [2 puntos]

**Respuesta:**

In [2]:
# instalar pycaret
#!pip install pycaret

In [3]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [4]:
datos = pd.read_csv("stats_players.csv", sep=",")

label = pd.DataFrame(~datos["National_Position"].isna())
label.value_counts()

X = datos.drop(columns=["National_Position"])
X.drop(columns=["Nationality"], inplace=True)
X.set_index("Name", inplace=True)
Y = label.copy()

In [5]:
X.columns = X.columns.str.replace(" ", "_")

Hay 1075 seleccionados nacionales y 16513 futbolistas no seleccionados, hay un gran desbalanze entre las clases.

In [6]:
X.columns

Index(['Club_Position', 'Height', 'Weight', 'Preffered_Foot', 'Age',
       'Work_Rate', 'Weak_foot', 'Skill_Moves', 'Ball_Control', 'Dribbling',
       'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Aggression',
       'Reactions', 'Interceptions', 'Vision', 'Composure', 'Crossing',
       'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed', 'Stamina',
       'Strength', 'Balance', 'Agility', 'Jumping', 'Heading', 'Shot_Power',
       'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties',
       'Volleys'],
      dtype='object')

In [7]:
# Definicion las variables que pasarán por cada pipeline
numerical_columns = [col for col in X.columns if pd.api.types.is_numeric_dtype(X[col])]
categorical_columns = [col for col in X.columns if col not in numerical_columns]

# Definicion del pipeline
numeric_transformations = (
    Pipeline(steps=[
        ('minmax', StandardScaler()),
    ])
)

# Definición del pipeline
categorical_transformations = (
    Pipeline(steps=[
        ('onehot', OneHotEncoder(sparse_output=False)),
    ])
)

# ColumnTransformer
col_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', categorical_transformations, categorical_columns)
                                        ],
                                        verbose_feature_names_out=False)

col_transformer.set_output(transform='pandas')



# Aplicamos ColumnTransformer a los datos
#df = col_transformer.fit_transform(X)

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('minmax', StandardScaler())]),
                                 ['Height', 'Weight', 'Age', 'Weak_foot',
                                  'Skill_Moves', 'Ball_Control', 'Dribbling',
                                  'Marking', 'Sliding_Tackle',
                                  'Standing_Tackle', 'Aggression', 'Reactions',
                                  'Interceptions', 'Vision', 'Composure',
                                  'Crossing', 'Short_Pass', 'Long_Pass',
                                  'Acceleration', 'Speed', 'Stamina',
                                  'Strength', 'Balance', 'Agility', 'Jumping',
                                  'Heading', 'Shot_Power', 'Finishing',
                                  'Long_Shots', 'Curve', ...]),
                                ('categorical',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(sparse_output=False))]),
                                 ['Club_Position', 'Preffered_Foot',
                                  'Work_Rate'])],
                  verbose_feature_names_out=False)

Todas las variables numéricas son no negativas, luego, ocupamos MinMaxScaler ya que preserva esta propiedad.

### 1.2 Entrenamiento [3 puntos]

Ahora, vamos a entrenar los pipelines generados en los pasos anteriores. Para esto, debe realizar las siguientes tareas:

1. Separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio). En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`). [1 puntos]


2. Defina un pipeline llamado `pipeline_xgboost` y otro llamado `pipeline_lightgbm`. Estos pipelines deben tener el mismo ColumnTransformer definido en la sección de preprocesamiento, pero deben variar los clasificadores de acuerdo al nombre de cada pipeline. [1 puntos]

3. Entrene los pipelines. [1 punto]



**Respuesta:**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.2, random_state=42, stratify=label)

In [9]:
from gc import enable
pipeline_xgboost =  (Pipeline(steps=[("col_transformer",  col_transformer),
                                   ("classify",XGBClassifier())]))
pipeline_lightgbm = (Pipeline(steps=[("col_transformer",  col_transformer),
                                   ("classify", LGBMClassifier())]))

In [10]:
xgbclass = pipeline_xgboost.fit(X_train, y_train)
lgbclass = pipeline_lightgbm.fit(X_train,y_train)

c:\Users\esteb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\esteb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 860, number of negative: 13210
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001558 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2532
[LightGBM] [Info] Number of data points in the train set: 14070, number of used features: 70
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.061123 -> initscore=-2.731797
[LightGBM] [Info] Start training from score -2.731797


### 1.3 Resultados [6 puntos]

1. Calcule las métricas accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional) para evaluar el rendimiento de los distintos modelos. Verifique sus resultados usando `classification_report`. [2 puntos]

2. Explique qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional) y cómo influye la cantidad de ejemplos por clase en los resultados obtenidos. [2 puntos]

3. Explique qué métrica le parece más adecuada y concluya qué modelo tiene un mejor desempeño. [2 puntos]

**Respuesta:**

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

# Ajustamos y_test por si todavía está como DataFrame de una columna
y_test = y_test.values.ravel()

# === Ejemplo con XGBoost ===
y_pred_xgb = xgbclass.predict(X_test)

acc_xgb = accuracy_score(y_test, y_pred_xgb)
prec_xgb = precision_score(y_test, y_pred_xgb, pos_label=1)  # precisión de la clase positiva
rec_xgb = recall_score(y_test, y_pred_xgb, pos_label=1)      # recall de la clase positiva

print("=== XGBoost ===")
print(f"Accuracy: {acc_xgb:.4f}")
print(f"Precisión (clase positiva): {prec_xgb:.4f}")
print(f"Recall (clase positiva): {rec_xgb:.4f}")
print("\nClassification report:\n", classification_report(y_test, y_pred_xgb))

# === Ejemplo con LightGBM ===
y_pred_lgb = lgbclass.predict(X_test)

acc_lgb = accuracy_score(y_test, y_pred_lgb)
prec_lgb = precision_score(y_test, y_pred_lgb, pos_label=1)
rec_lgb = recall_score(y_test, y_pred_lgb, pos_label=1)

print("\n=== LightGBM ===")
print(f"Accuracy: {acc_lgb:.4f}")
print(f"Precisión (clase positiva): {prec_lgb:.4f}")
print(f"Recall (clase positiva): {rec_lgb:.4f}")
print("\nClassification report:\n", classification_report(y_test, y_pred_lgb))

=== XGBoost ===
Accuracy: 0.9366
Precisión (clase positiva): 0.4444
Recall (clase positiva): 0.1488

Classification report:
               precision    recall  f1-score   support

       False       0.95      0.99      0.97      3303
        True       0.44      0.15      0.22       215

    accuracy                           0.94      3518
   macro avg       0.70      0.57      0.59      3518
weighted avg       0.92      0.94      0.92      3518


=== LightGBM ===
Accuracy: 0.9395
Precisión (clase positiva): 0.5192
Recall (clase positiva): 0.1256

Classification report:
               precision    recall  f1-score   support

       False       0.95      0.99      0.97      3303
        True       0.52      0.13      0.20       215

    accuracy                           0.94      3518
   macro avg       0.73      0.56      0.59      3518
weighted avg       0.92      0.94      0.92      3518



El accuracy alto indica que la mayoría de las veces que el modelo predice un label, este acierta, pero la precision y el recall al ser bajos (para la clase positiva) indican que el accuracy elevado se debe a que los modelos predicen casi siempre la clase negativa y como hay un desbalanceo de clases importante se infla el accuracy.

Para el contexto tan desbalanceado la metrica más importante es el recall, ya que se desea ver de los jugadores seleccionados, cuantos detecto el modelo. Por lo que en este contexto el modelo XGBoost es ligeramente mejor, aunque ninguno tiene un buen rendimiento.

## 2. Predicción de posiciones de jugadores [4 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="300">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas.

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características.

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo `Club_Position`, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF']
central_ataque = ['RW', 'CAM', 'LW']
central = ['RM', 'CM', 'LM']
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

La elección del clasificador se justificar en base a la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y se deben comentar los resultados obtenidos en la clasificación.

**Tareas:** [1 punto por tarea]

1. En un nuevo dataframe, aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección y guárdelos en la variable `label`.
2. Cuente cuántos por clase quedan.
3. Entrene el nuevo pipeline y ejecute una evaluación de este.  
4. Comente los resultados obtenidos.

**Respuesta:**

In [12]:
club_position_mapping = {
    "ataque": ["ST", "CF"],
    "central_ataque": ["RW", "CAM", "LW"],
    "central": ["RM", "CM", "LM"],
    "central_defensa": ["RWB", "CDM", "LWB"],
    "defensa": ["RB", "CB", "LB"],
    "arquero": ["GK"]
}

position_to_category = {pos: cat for cat, positions in club_position_mapping.items() for pos in positions}

new_feature_df = X.copy()
new_feature_df["label"] = new_feature_df["Club_Position"].map(position_to_category) #.fillna(new_feature_df["Club_Position"])
new_feature_df.dropna(subset=["label"], inplace=True)

X = new_feature_df.drop(columns=["Club_Position", "label"])
label = new_feature_df["label"]

In [13]:
label.value_counts()

label
defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: count, dtype: int64

In [14]:
# Definicion las variables que pasarán por cada pipeline
numerical_columns = [col for col in X.columns if pd.api.types.is_numeric_dtype(X[col])]
categorical_columns = [col for col in X.columns if col not in numerical_columns]

# Definicion del pipeline
numeric_transformations = (
    Pipeline(steps=[
        ('minmax', StandardScaler()),
    ])
)

# Definición del pipeline
categorical_transformations = (
    Pipeline(steps=[
        ('onehot', OneHotEncoder(sparse_output=False)),
    ])
)

# ColumnTransformer
col_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', categorical_transformations, categorical_columns)
                                        ],
                                        verbose_feature_names_out=False)

col_transformer.set_output(transform='pandas')

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('minmax', StandardScaler())]),
                                 ['Height', 'Weight', 'Age', 'Weak_foot',
                                  'Skill_Moves', 'Ball_Control', 'Dribbling',
                                  'Marking', 'Sliding_Tackle',
                                  'Standing_Tackle', 'Aggression', 'Reactions',
                                  'Interceptions', 'Vision', 'Composure',
                                  'Crossing', 'Short_Pass', 'Long_Pass',
                                  'Acceleration', 'Speed', 'Stamina',
                                  'Strength', 'Balance', 'Agility', 'Jumping',
                                  'Heading', 'Shot_Power', 'Finishing',
                                  'Long_Shots', 'Curve', ...]),
                                ('categorical',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(sparse_output=False))]),
                                 ['Preffered_Foot', 'Work_Rate'])],
                  verbose_feature_names_out=False)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.2, random_state=42, stratify=label)

In [16]:
pipeline_xgboost =  (Pipeline(steps=[("col_transformer",  col_transformer),
                                   ("classify", XGBClassifier())]))
pipeline_lightgbm = (Pipeline(steps=[("col_transformer",  col_transformer),
                                   ("classify", LGBMClassifier())]))

In [17]:
from sklearn.preprocessing import LabelEncoder

# Codificar y_train
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)  # para evaluación

In [18]:
xgbclass = pipeline_xgboost.fit(X_train, y_train_encoded)
lgbclass = pipeline_lightgbm.fit(X_train,y_train_encoded)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000443 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2342
[LightGBM] [Info] Number of data points in the train set: 3151, number of used features: 43
[LightGBM] [Info] Start training from score -1.828938
[LightGBM] [Info] Start training from score -2.214833
[LightGBM] [Info] Start training from score -1.469303
[LightGBM] [Info] Start training from score -1.913438
[LightGBM] [Info] Start training from score -2.937481
[LightGBM] [Info] Start training from score -1.205349
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGB

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

# Ajustamos y_test por si todavía está como DataFrame de una columna
#y_test_encoded = y_test_encoded.values.ravel()

# === Ejemplo con XGBoost ===
y_pred_xgb = xgbclass.predict(X_test)
y_pred_decoded = le.inverse_transform(y_pred_xgb)
print("=== XGBoost ===")
print("\nClassification report:\n", classification_report(y_test, y_pred_decoded))

# === Ejemplo con LightGBM ===
y_pred_lgb = lgbclass.predict(X_test)
y_pred_lgb_decoded = le.inverse_transform(y_pred_lgb)
print("=== LGBM ===")
print("\nClassification report:\n", classification_report(y_test, y_pred_lgb_decoded))

=== XGBoost ===

Classification report:
                  precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       126
         ataque       0.76      0.83      0.79        86
        central       0.59      0.64      0.62       182
 central_ataque       0.51      0.39      0.44       116
central_defensa       0.35      0.14      0.20        42
        defensa       0.84      0.95      0.89       236

       accuracy                           0.75       788
      macro avg       0.68      0.66      0.66       788
   weighted avg       0.72      0.75      0.73       788

=== LGBM ===

Classification report:
                  precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       126
         ataque       0.75      0.81      0.78        86
        central       0.58      0.63      0.60       182
 central_ataque       0.49      0.36      0.42       116
central_defensa       0.33      0.12      0.18        42
     

A pesar de que el accuracy es menor que en el caso anterior para ambos modelos, el recall y la precision son mucho mejores, aunque depende de la clase, ya que por ejemplo el central_defensa pareciera la clase más díficil de predecir.

## 3. Predicciones de Seleccionados Nacionales para el Jere Klein [30 puntos]

<center>
<img src='https://www.radioactiva.cl/wp-content/uploads/2024/04/Jere-Klein-1-768x432.webp' width=500 />

Después de alcanzar la fama como cantante urbano, Jere Klein decide explorar una nueva faceta. Con su amor por el fútbol y convencido de que los artistas urbanos poseen un talento y versatilidad excepcionales, Jere se embarca en un proyecto innovador: desarrollar un sistema de inteligencia artificial capaz de identificar a jugadores que tienen potencial para convertirse en futbolistas profesionales. Su teoría es que muchos artistas del género urbano chileno, con sus habilidades únicas y su disciplina, podrían destacarse también en el deporte. Con este sistema, Jere espera no solo abrir nuevas oportunidades para sus colegas artistas, sino también demostrar la amplia gama de talentos que pueden ofrecer.

### 2.1 ¿Qué modelo de árbol es más de "pana"? [10 puntos]

<center>
<img src='https://64.media.tumblr.com/39189215a7d3d96823cb359f35b44e05/tumblr_psmrhrR3Xw1qf5hjqo4_540.gif' width=300 />


**Tareas**


1. Considerando el la variable llamada `label` creada en la sección 1.1. Para determinar cuál modelo de árbol sería más adecuado para la tarea en cuestión, utilice PyCaret. Este deberá centrarse exclusivamente en modelos de tipo árbol. Jere ha especificado que busca un modelo que tome decisiones rápidamente y que tenga una baja tasa de falsos positivos, ya que planea invertir en estos jugadores. [3 puntos]

Para la comparación, utilice los siguientes modelos:

```python
['et', 'rf', 'dt', 'xgboost', 'lightgbm', 'catboost']
```

2. Explique en brevemente que son los modelos de la siguiente lista `['et', 'rf', 'dt']` y como funcionan. [3 punto]

3. Tras realizar la comparación de modelos, seleccione aquel que muestre el mejor rendimiento en términos de velocidad y precisión, especialmente en la reducción de falsos positivos. Utilice la función `evaluate_model` de PyCaret para revisar y analizar los resultados obtenidos en los siguientes aspectos:

  - **Confusión Matrix**: ¿Cómo se encuentran la tasa de verdaderos positivos y verdaderos negativos?
  - **Threshold**: ¿Es acaso el umbral por defecto del modelo el mejor para las predicciones?
  - **Feature Importance**: ¿Cuáles son las variables con mejor desempeño? ¿A qué podría deberse esto?
  - **Learning Curve**: ¿El modelo presenta algún problema?

  [4 puntos]

**Respuesta**

In [27]:
from pycaret.datasets import get_data
from pycaret.classification import *
import os

os.environ["PYCARET_CUSTOM_LOGGING_LEVEL"] = "CRITICAL"

# 2. Importar lo necesario
from pycaret.classification import setup, compare_models

# 3. Crear el setup de PyCaret
clf_setup = setup(
    data=X_train.assign(label=y_train_encoded),  # Combinar X_train con la variable objetivo
    target='label',                      # Nombre de la variable objetivo
    session_id=42,                       # Semilla reproducible
    use_gpu=True,                        # Si tienes GPU disponible
    fold=5,                              # Validación cruzada de 5 folds
    verbose=True,
    index=False
)

# 4. Comparar solo los modelos especificados
best_tree_model = compare_models(
    include=['et', 'rf', 'dt', 'xgboost', 'lightgbm', 'catboost'],
    sort='Precision'   # Puedes priorizar precisión (para reducir falsos positivos)
)

best_tree_model

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3080 Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0,500000 -> initscore=0,000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leav

,Description,Value
0,Session id,42
1,Target,label
2,Target type,Multiclass
3,Original data shape,"(3177, 36)"
4,Transformed data shape,"(3177, 44)"
5,Transformed train set shape,"(2223, 44)"
6,Transformed test set shape,"(954, 44)"
7,Numeric features,33
8,Categorical features,2
9,Preprocess,True


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3080 Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0,500000 -> initscore=0,000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leav

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.7305,0.9167,0.7305,0.7167,0.7028,0.6529,0.6596,0.9320
catboost,CatBoost Classifier,0.7278,0.9176,0.7278,0.7159,0.7118,0.6519,0.6551,9.8640
rf,Random Forest Classifier,0.7305,0.9196,0.7305,0.7115,0.7046,0.6535,0.6595,0.2620
xgboost,Extreme Gradient Boosting,0.7224,0.9144,0.7224,0.7075,0.7082,0.6458,0.6482,2.2140
lightgbm,Light Gradient Boosting Machine,0.7157,0.9131,0.7157,0.7019,0.6982,0.6361,0.6392,7.2260
dt,Decision Tree Classifier,0.6181,0.7648,0.6181,0.6278,0.6216,0.5211,0.5217,0.1040


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     monotonic_cst=None, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=42, verbose=0,
                     warm_start=False)

Respuesta a pregunta 2:

1) El Decision Tree (dt) es un modelo de árbol de decisión básico que divide los datos basandose en características que maximizan la separación entre clases. Es fácil de interpretar pero es muy dado a sobreajustarse a los datos de entrenamiento, en general este modelo es utilizado como base para modelos más complejos.

2) El Random Forest (rf) es un conjunto de árboles de decisión entrenados con diferentes subconjuntos de los datos de entrenamiento. Cada árbol vota por una clase y la clase con más votos es la predicción final. Esto reduce el sobreajuste y mejora la precisión.

3) El Extra Trees (et) es similar al Random Forest, pero introduce más aleatoriedad en los cortes de los árboles y en la selección de características. Esto puede mejorar la diversidad entre los árboles y la velocidad de entrenamiento.

In [29]:
# pregunta 3
# el mejor modelo es extra trees (et) con una precision de 0.71
evaluate_model(best_tree_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

### 2.2 Reducción de dimensionalidad [14 puntos]

<center>
<img src='https://i.kym-cdn.com/photos/images/original/002/258/560/668.gif' width=400 />

A pesar de los resultados obtenidos previamente, el manager de Jere ha solicitado el entrenamiento de un modelo de XGBoost utilizando los datos disponibles. Además, se debe proceder a realizar una reducción de dimensionalidad basada en la importancia de las características.

Para llevar a cabo esta tarea:

1. Inicie entrenando un modelo XGBoost con todas las características disponibles. [2 puntos]

2. Una vez el modelo esté entrenado, evalúe y clasifique las características según su importancia de forma descendente. [2 puntos]

3. Utilice esta clasificación para ejecutar una búsqueda recursiva de eliminación de características, eliminando progresivamente las menos importantes y evaluando el impacto en el desempeño del modelo hasta identificar las N características más críticas. [2 puntos]

4. Con este conjunto reducido de características, entrene un nuevo modelo y evalúe su rendimiento. [2 puntos]

5. Posteriormente, responda a las siguientes preguntas para una comprensión más profunda de los cambios y beneficios:

  - ¿El rendimiento del modelo con las características seleccionadas es similar al del modelo original? ¿Cómo se comparan en términos de precisión y robustez? [2 puntos]
  - ¿Cuáles son los beneficios potenciales de eliminar variables del modelo? Considere factores como la simplificación del modelo, reducción del tiempo de entrenamiento, y mejora en la capacidad de generalización. [2 puntos]
  - Comente si el modelo con menor dimensionalidad es más sencillo de explicar. Explique brevemente por qué la eliminación de ciertas características puede facilitar la comprensión y la explicación del comportamiento del modelo. [2 puntos]

Notar que con esta metodologia buscamos encontrar un punto entermedio entre número de festures y desempeño. por esto, si observa que al aumentar festires el aumento es despreciable, puede no considerar agregar más features a su modelo.

**Respuesta**

### 2.3 Calibración Probabilistica [6 puntos]

<center>
<img src='https://media2.giphy.com/media/l2Je4Ku0Cx292KWv6/200w.gif?cid=6c09b952y0sihtq9tb6sz8j2023x3zxxp3qx1ocgonkpkblj&ep=v1_gifs_search&rid=200w.gif&ct=g' width=400 />

Para lograr modelos más modulares, se recomienda realizar una calibración del modelo entrenado anteriormente, con el objetivo de obtener salidas que reflejen mayor modularidad.

1. Se solicita que utilice un método de calibración que asegure que las probabilidades generadas incrementen de manera monótona. Una métrica ampliamente utilizada para evaluar la precisión de la calibración de un modelo es el Brier Score. Calcule el Brier Score para el modelo tanto antes como después de la calibración. Esto le permitirá realizar una comparación cuantitativa y determinar si la calibración ha mejorado el rendimiento del modelo. Para más información sobre el Brier Score, puede consultar el siguiente enlace: [Scikit-Learn - Brier Score Loss](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.brier_score_loss.html). [3 puntos]

2. Tras la calibración, examine y comente los resultados obtenidos. A su análisis añada una comparación visual de las ideales versus las salidas del modelo original (sin calibrar) y del modelo calibrado. [3 puntos]

**Respuesta**

Mucho éxito!

<center>
<img src="https://i.pinimg.com/originals/55/3d/42/553d42bea9b10e0662a05aa8726fc7f4.gif" width=300>